In [1]:
import os
import sys
sys.path.append('uplift-campaign/')

import pandas as pd
import numpy as np
import dask.dataframe as dd
import pickle
import json
import sklearn.base as skbase
import sklearn.pipeline as skpipe
import datetime

In [2]:
from upcampaign.utils.flow import Flow

In [3]:
from upcampaign.utils.data import dump_pickle, load_pickle

In [4]:
# сохранение промежуточных результатов (x, x+y, x+y+z)

In [5]:
ls  

artifacts/  configs/  flow_example.ipynb  learn.ipynb  runs/  uplift-campaign/


In [12]:
class SumFlow(Flow):
    def __init__(self, x: float, y: float, z: float, **kwargs):
        super(SumFlow, self).__init__(**kwargs)
        self.x = x
        self.y = y
        self.z = z
        
        self.result = 0
        self.result_x_path = 'result_x.pickle'
        self.result_xy_path = 'result_xy.pickle'
        self.result_xyz_path = 'result_xyz.pickle'

        self.set_stages([
            'add_x',
            'add_y',
            'add_z',
        ])
        
    def _run_stage_add_x(self):
        print('I add x')
        self.result += self.x
        dump_pickle(self.result, os.path.join(self.get_directory(), self.result_x_path))
        
    def _restore_stage_add_x(self):
        print('I restore x')
        self.result = load_pickle(self.result_x_path)
        
    def _run_stage_add_y(self):
        print('I add y')
        self.result += self.y
        dump_pickle(self.result, os.path.join(self.get_directory(), self.result_xy_path))
    
    def _restore_stage_add_y(self):
        print('I restore y')
        self.result = load_pickle(self.result_xy_path)

    def _run_stage_add_z(self):
        print('I add z')
        self.result += self.z
        dump_pickle(self.result, os.path.join(self.get_directory(), self.result_xyz_path))
    
    def _restore_stage_add_z(self):
        print('I restore z')
        self.result = load_pickle(self.result_xyz_path)


In [13]:
ls

artifacts/  configs/  flow_example.ipynb  learn.ipynb  runs/  uplift-campaign/


In [14]:
flow = SumFlow(1, 2, 4, run_id='test_sum_flow_n1', runs_root_path='runs')

In [15]:
! ls runs/

test_sum_flow_n1  test_sum_flow_n2  test_sum_flow_n3


In [16]:
! ls runs/test_sum_flow_n1

_stage_add_x_completed.flg  _stage_add_z_completed.flg
_stage_add_y_completed.flg


In [18]:
flow.run()

I add x
I add y
I add z


In [19]:
flow = SumFlow(1, 2, 4, run_id='test_sum_flow_n2', runs_root_path='runs')
добавляем raise ValueError

In [20]:
! ls runs/

test_sum_flow_n1  test_sum_flow_n2  test_sum_flow_n3


In [21]:
! ls runs/test_sum_flow_n2

_stage_add_x_completed.flg  _stage_add_y_completed.flg


In [22]:
flow.run()

I restore y


FileNotFoundError: [Errno 2] No such file or directory: 'result_xy.pickle'

In [30]:
! ls runs/test_sum_flow_n2

_stage_add_x_completed.flg _stage_add_y_completed.flg


In [39]:
flow = SumFlow(1, 2, 4, run_id='test_sum_flow_n3', runs_root_path='runs')

In [34]:
! ls runs/

n1_try1          test_sum_flow_n1 test_sum_flow_n2 test_sum_flow_n3


In [35]:
! ls runs/test_sum_flow_n3

In [36]:
flow.run()

I add x
I add y
I add z


ValueError: 4

In [37]:
! ls runs/test_sum_flow_n3

_stage_add_x_completed.flg result_x.pickle
_stage_add_y_completed.flg result_xy.pickle


In [40]:
flow = SumFlow(1, 2, 4, run_id='test_sum_flow_n3', runs_root_path='runs')

In [41]:
flow.run()

I restore y
I add z
